In [9]:
import numpy as np
import itertools

def initialize_parameters(N, K, seed=None):
    #Initialize parameters randomly

    if seed is not None:
        np.random.seed(seed)

    pi = np.random.dirichlet(alpha=np.ones(K))    

    check = np.allclose(sum(pi), 1, atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")


    # Transition matrix. it has K rows and K columns. In particular
    # the sum of probabilities from the same initial state should be 1
    gamma = np.random.dirichlet(alpha=np.ones(K), size=K).T

    sum_columns = np.sum(gamma, axis = 0)
    check = np.allclose(sum_columns, np.ones(K), atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")


    # I initialize the r matrix in this in order to satisfy the boundary condition (sum of 
    # columns must be 1 foreach column)
    r = np.random.dirichlet(alpha=np.ones(N + 1), size=K).T

    sum_columns = np.sum(r, axis = 0)
    check = np.allclose(sum_columns, np.ones(K), atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")

    return pi, gamma, r

def generate_hmm_data(T, N, K, pi, gamma, r):
    z = np.zeros(T, dtype=int)  
    y = np.zeros(T, dtype=int)  

    # Initialization t=0. I extract the value of K following the init prob
    z[0] = np.random.choice(K, p=pi)

    #data generation
    for t in range(T):
        if t > 0:
            # I take the column of gamma corresponding to z[t-1] and extract from this distribution
            z[t] = np.random.choice(K, p=gamma[:, z[t-1]])

        # I take the column of r corresponding to z[t] and extract from this prob distribution
        y[t] = np.random.choice(N + 1, p=r[:, z[t]])

    return y, z

def likelihood_comp(T, K, pi, gamma, r, y):

    likelihood = 0
    # all possible sequences z
    all_sequences = itertools.product(range(K), repeat=T)

    for z in all_sequences: 

        p_1 = 0
        p_2 = pi[z[0]]

        for t in range(T-1):
            p_1 *= r[y[t], z[t]]
            p_2 *= gamma[z[t], z[t+1]]

        p_1 *= r[y[T-1], z[T-1]]

        likelihood += (p_1*p_2)

    return likelihood

    


N = 4  # neurons
K = 3  # states
T = 100  # time steps
seed = None #1234  

#input: initialization of the parameters for the HMM
pi, gamma, r = initialize_parameters(N, K, seed)

print("pi = ", pi)
print("gamma = ", gamma)
print("r = ", r)

# print(r[y[0]])

#output: generation of the observed and hidden data
y, z = generate_hmm_data(T, N, K, pi, gamma, r)

print("y:", y)
print("z:", z)

# likelihood = likelihood_comp(T, K, pi, gamma, r, y)
# print(likelihood)

pi =  [0.04203132 0.20931219 0.74865649]
gamma =  [[0.67091829 0.47374936 0.12633601]
 [0.17980311 0.15155185 0.15009608]
 [0.1492786  0.37469879 0.72356792]]
r =  [[0.17206994 0.0431599  0.21443106]
 [0.22476455 0.0703177  0.09739114]
 [0.16494576 0.34443665 0.15214959]
 [0.12081422 0.41704232 0.45246401]
 [0.31740552 0.12504343 0.08356421]]
y: [2 0 0 3 4 3 3 2 1 3 4 3 0 3 4 3 3 1 3 1 3 0 0 1 0 2 0 2 3 4 1 4 1 0 3 2 4
 1 4 4 2 0 2 2 3 3 3 2 0 1 3 4 1 4 3 3 0 3 0 1 4 4 2 4 3 3 0 3 3 4 2 3 2 3
 3 4 2 4 3 0 0 3 4 0 3 2 1 1 2 0 0 2 0 1 3 2 3 3 0 3]
z: [1 0 0 0 0 2 2 2 2 2 2 1 2 2 2 0 0 2 2 1 1 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0
 1 0 1 1 2 2 2 2 2 2 2 2 2 2 2 0 0 2 2 1 1 0 1 0 0 0 0 2 2 2 1 0 1 2 2 2 2
 2 2 0 2 2 2 2 2 2 0 0 0 0 2 1 2 2 0 0 0 1 0 1 0 0 1]
